In [1]:
import numpy as np
import pandas as pd

In [9]:
dtype={'SRNumber': np.str_,
       'CreatedDate': np.str_,
       'UpdatedDate': np.str_,
       'ActionTaken': np.str_,
       'Owner': np.str_,
       'RequestType': np.str_,
       'Status': np.str_,
       'RequestSource': np.str_,
       'MobileOS': np.str_,
       'Anonymous': np.str_,
       'AssignTo': np.str_,
       'ServiceDate': np.str_,
       'ClosedDate': np.str_,
       'AddressVerified': np.str_,
       'ApproximateAddress': np.str_,
       'Address': np.str_,
       'HouseNumber': np.str_,
       'Direction': np.str_,
       'StreetName': np.str_,
       'Suffix': np.str_,
       'ZipCode': np.str_,
       'Latitude': np.float64,
       'Longitude': np.float64, 
       'Location': np.str_,
       'TBMPage': np.str_,
       'TBMColumn': np.str_,
       'TBMRow': np.str_,
       'APC': np.str_,
       'CD': np.str_,
       'CDMember': np.str_,
       'NC': np.str_,
       'NCName': np.str_,
       'PolicePrecinct': np.str_,}

data = pd.read_csv("MyLA311_Service_Request_Data_2016.csv",dtype=dtype, usecols=dtype)
print data.head(1)

     SRNumber             CreatedDate             UpdatedDate ActionTaken  \
0  1-88224881  01/01/2016 08:12:35 AM  01/07/2016 07:39:04 AM  SR Created   

  Owner             RequestType  Status RequestSource MobileOS Anonymous  \
0   BOS  Illegal Dumping Pickup  Closed          Call      NaN         N   

       ...                              Location TBMPage TBMColumn TBMRow  \
0      ...        (34.051491916, -118.366682002)     633         A      4   

           APC  CD            CDMember  NC       NCName PolicePrecinct  
0  Central APC  10  Herb J. Wesson Jr.  60  P.I.C.O. NC       WILSHIRE  

[1 rows x 33 columns]


In [10]:

from datetime import datetime

In [11]:
requestType = "Bulky Items,Dead Animal Removal,Electronic Waste,Feedback,Graffiti Removal,Homeless Encampment,Illegal Dumping Pickup,Metal/Household Appliances,Multiple Streetlight Issue,Report Water Waste,Single Streetlight Issue".split(",")

In [12]:
print(requestType)

['Bulky Items', 'Dead Animal Removal', 'Electronic Waste', 'Feedback', 'Graffiti Removal', 'Homeless Encampment', 'Illegal Dumping Pickup', 'Metal/Household Appliances', 'Multiple Streetlight Issue', 'Report Water Waste', 'Single Streetlight Issue']


In [15]:
point_col = []
month_col = []
request_col = []
time_col = []
for i in np.arange(len(data)):
    if(not np.isnan(data["Latitude"][i])):
        point_col.append(geometry.Point(data["Latitude"][i],data["Longitude"][i]))
        t1 = datetime.strptime(data["CreatedDate"][i], '%m/%d/%Y %I:%M:%S %p')
        t2 = datetime.strptime(data["UpdatedDate"][i], '%m/%d/%Y %I:%M:%S %p')
        month_col.append(t1.month)
        time_col.append((t2-t1).days)
        request_col.append(data["RequestType"][i])

In [16]:
with open("coorList.txt", "r") as f:
    temp = f.read()
temp = temp[:-1].split("\n")
polygon_list = [geometry.Polygon([(np.float64(j.split(",")[1]), np.float64(j.split(",")[0])) for j in i.split(" ")[:-1]]) for i in temp]
print len(polygon_list)

114


In [17]:
data2 = pd.read_csv("health_data.csv", names=['year','latitude','a','longitude','b','population','value'])
health_latitude = [np.float64(i[9:]) for i in data2["latitude"]][2:]
health_longitude = [np.float64(i[9:]) for i in data2["longitude"]][2:]
health_point = [geometry.Point(health_latitude[i],health_longitude[i]) for i in np.arange(len(health_latitude))]
population_col = [i for i in data2["population"]][2:]
value_col = [i for i in data2["value"]][2:]

for i in np.arange(len(value_col)):
    if("Empty" in value_col[i]):
        value_col[i] = 0
    else:
        value_col[i] = int(float(value_col[i])/100*population_col[i])

In [18]:
health_dataset = np.zeros((len(polygon_list),2))
for i in np.arange(len(health_point)):
    for j in np.arange(len(polygon_list)):
        if(polygon_list[j].contains(health_point[i])):
            health_dataset[j,0]+=population_col[i]
            health_dataset[j,1]+=value_col[i]

In [20]:
print health_dataset
header = list(requestType)
header.append("population")
header.append("value")
print header

[[  1.75250000e+04   3.19400000e+03]
 [  3.11870000e+04   4.73400000e+03]
 [  1.80340000e+04   3.04500000e+03]
 [  1.08470000e+04   1.28300000e+03]
 [  3.03740000e+04   4.96400000e+03]
 [  8.26100000e+03   6.98000000e+02]
 [  1.21910000e+04   1.10900000e+03]
 [  2.09980000e+04   1.82100000e+03]
 [  4.04300000e+03   3.31000000e+02]
 [  9.11930000e+04   1.89240000e+04]
 [  3.25840000e+04   2.47300000e+03]
 [  2.18300000e+04   4.69500000e+03]
 [  5.88020000e+04   8.15900000e+03]
 [  3.28900000e+03   3.19000000e+02]
 [  4.34470000e+04   9.81100000e+03]
 [  2.42800000e+03   2.62000000e+02]
 [  3.95500000e+04   4.17900000e+03]
 [  8.00000000e+00   0.00000000e+00]
 [  1.42690000e+04   2.49100000e+03]
 [  9.02300000e+03   7.71000000e+02]
 [  2.07430000e+04   3.49600000e+03]
 [  9.34200000e+03   1.65900000e+03]
 [  2.76140000e+04   3.33100000e+03]
 [  4.41700000e+04   7.65200000e+03]
 [  3.16100000e+04   3.32600000e+03]
 [  6.73930000e+04   1.13860000e+04]
 [  3.42520000e+04   4.46900000e+03]
 

In [34]:
dataset = np.zeros((len(health_dataset)*12,len(header)))
for i in np.arange(len(point_col)):
    for j in np.arange(len(polygon_list)):
        if(polygon_list[j].contains(point_col[i]) and request_col[i] in requestType):
            dataset[j*12+month_col[i]-1,header.index(request_col[i])]+=1

In [38]:
for i in np.arange(len(health_dataset)):
    for j in np.arange(12):
        for k in range(header.index("population")-1):
            dataset[i*12+j,k] = dataset[i*12+j,k]/health_dataset[i][0]
        dataset[i*12+j,header.index("population")] = health_dataset[i][0]
        dataset[i*12+j,header.index("value")] = health_dataset[i][1]/health_dataset[i][0]*100
max_v = max(dataset[:,12])
print max_v
for i in np.arange(len(dataset)):
    dataset[i,12] = (max_v-dataset[i,12])/max_v*10
print dataset

25.0
[[  1.86586647e-15   8.48121124e-17   1.59022711e-16 ...,   0.00000000e+00
    1.75250000e+04   2.70984308e+00]
 [  1.35699380e-15   6.36090843e-17   1.48421197e-16 ...,   0.00000000e+00
    1.75250000e+04   2.70984308e+00]
 [  1.62203165e-15   6.36090843e-17   2.01428767e-16 ...,   2.00000000e+00
    1.75250000e+04   2.70984308e+00]
 ..., 
 [  5.48534250e-17   2.58858860e-18   5.54697556e-18 ...,   2.20000000e+01
    5.33690000e+04   6.09211340e+00]
 [  5.04158446e-17   1.72572573e-18   5.30044332e-18 ...,   1.60000000e+01
    5.33690000e+04   6.09211340e+00]
 [  5.00460462e-17   3.20491921e-18   6.16330618e-18 ...,   1.60000000e+01
    5.33690000e+04   6.09211340e+00]]


In [43]:
import math
for i in np.arange(len(dataset)):
    dataset[i,12] = math.ceil((dataset[i,12]))

In [44]:
np.savetxt("data.csv", dataset, delimiter=",")

In [45]:
import scipy.stats as stats

In [48]:
with open("test_scores.txt", "r") as f:
    scores = f.read().split("\n")
score = np.zeros(114)
for i in range(114):
    for j in np.arange(12):
        score[i]+=float(scores[i*12+j])
    score[i] = np.int64(score[i]/12)

t_stat, p_val = stats.ttest_ind(score, dataset[:,8], equal_var=False)

In [49]:
t_stat

21.520556418625151

In [50]:
p_val

8.9180701404742706e-42

In [51]:
from sklearn.ensemble import ExtraTreesClassifier

In [52]:
data = dataset[:,:-1]
target = dataset[:,-1]
model = ExtraTreesClassifier()
model.fit(data, target)
feature_scores = model.feature_importances_
fs = np.sort(feature_scores)[::-1]
rank =[]
for i in fs:
    for j in range(len(feature_scores)):
        if(i == feature_scores[j]):
            rank.append(j)
for i in rank:
    print header[i]

population
Single Streetlight Issue
Bulky Items
Graffiti Removal
Illegal Dumping Pickup
Dead Animal Removal
Electronic Waste
Homeless Encampment
Report Water Waste
Metal/Household Appliances
Feedback
Multiple Streetlight Issue
Feedback
Multiple Streetlight Issue
